# FedLib: Simulating FedAvg using FedLib virtual Federated environment

## Importing supportive libaries
This notebook shows a demo on PyTorch back-end model impelementation.

In the very begining, we import the supporting libraries.

In [8]:
import torch
import numpy as np
import copy

from fedlib.simulator import BaseSimulator as Simulator
from fedlib.lib import Server, Client
from fedlib.networks import resnet20
from fedlib.lib.sampler import random_sampler
from fedlib.lib.algo.fedavg import Trainer 
from fedlib.datasets import partition_data, get_dataloader,get_client_dataloader


## Define arguments
Here we define arguments. To show an intuitive example, we show the demo store all the parameters in a dictionary in the following code block.
We also provide APIs for you create your arguments in a `*.yaml` file.

In [9]:
args = {}
args["n_clients"] = 100
args["device"] = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
args['sample_fn'] = random_sampler
args['trainer'] = Trainer()
args['communicator'] = None
args["test_dataset"] = None
args["partition"] = "noniid-labeldir"
args["dataset"] = "cifar10"
args["datadir"] = "./data"
args["beta"] = 0.5
args["batch_size"] = 64
args["global_model"] = resnet20()
args["lr"] = 0.01
args["optimizer"] = "SGD"
args["lr_scheduler"] = "ExponentialLR"

Load test dataset for server, and passing it as an argument

In [10]:
X_train, y_train, X_test, y_test, net_dataidx_map, traindata_cls_counts = partition_data(
    args["dataset"], args["datadir"], args['partition'], args['n_clients'], beta=args['beta'])
n_classes = len(np.unique(y_train))
train_dl_global, test_dl_global, train_ds_global, test_ds_global = get_dataloader(args["dataset"],
                                                                                    args["datadir"],
                                                                                      args["batch_size"],
                                                                                      32)
args["test_dataset"] = test_dl_global

Files already downloaded and verified
Files already downloaded and verified


INFO:root:Data statistics: {0: {0: 20, 1: 24, 2: 40, 3: 69, 4: 98, 5: 10, 7: 108, 8: 68, 9: 39}, 1: {0: 80, 1: 111, 2: 95, 3: 3, 4: 121, 5: 17, 6: 1, 8: 185}, 2: {0: 3, 1: 84, 2: 14, 3: 69, 4: 4, 5: 51, 6: 18, 7: 1, 8: 145, 9: 28}, 3: {0: 169, 1: 47, 2: 38, 3: 2, 4: 42, 5: 2, 6: 1, 7: 50, 9: 8}, 4: {0: 45, 1: 32, 2: 10, 3: 4, 4: 12, 5: 109, 6: 9, 7: 6, 8: 19, 9: 12}, 5: {0: 138, 1: 4, 2: 15, 3: 25, 4: 15, 5: 112, 6: 1, 7: 8, 8: 49, 9: 33}, 6: {0: 5, 1: 1, 2: 84, 3: 31, 4: 61, 5: 2, 6: 6, 7: 19, 8: 74, 9: 13}, 7: {0: 83, 1: 10, 2: 8, 3: 5, 4: 48, 5: 81, 6: 9, 7: 18, 8: 6, 9: 22}, 8: {0: 1, 1: 1, 2: 41, 3: 39, 4: 74, 5: 18, 6: 83, 7: 1, 8: 63, 9: 38}, 9: {0: 142, 1: 23, 2: 19, 3: 91, 4: 2, 5: 178, 6: 27, 7: 3, 8: 86}, 10: {0: 73, 1: 11, 2: 30, 3: 7, 5: 39, 6: 3, 7: 2, 8: 17, 9: 2}, 11: {0: 8, 2: 93, 3: 37, 4: 12, 5: 19, 6: 7, 7: 128, 8: 13}, 12: {0: 4, 1: 9, 2: 70, 3: 8, 4: 1, 5: 27, 7: 90, 8: 10, 9: 5}, 13: {0: 3, 1: 1, 2: 112, 3: 129, 4: 155, 5: 15, 6: 25, 7: 344}, 14: {0: 131, 1: 45, 

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


## Create server and clients objects
Here we use the arguments we defined before, and create server and clients.

In [11]:
server = Server(**args)
clients = {}

data_loaders = get_client_dataloader(args["dataset"], args["datadir"], args['batch_size'], 32, net_dataidx_map)


for id in range(args["n_clients"]):
    # dataidxs = net_dataidx_map[id]
    args["id"] = id
    # args["trainloader"], _, _, _ = get_dataloader(args["dataset"], args["datadir"], args['batch_size'], 32, dataidxs)
    args["trainloader"] = data_loaders[id]
    args["model"] = copy.deepcopy(args["global_model"])
    clients[id] = Client(**args)

criterion = torch.nn.CrossEntropyLoss()


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files alread

## Create simulator

Simulator simulates the virtual federated learning environments, and run server and clients on single device.

In [13]:
simulator = Simulator(server=server, clients=clients, communication_rounds=10,n_clients= 100,sample_rate=.1)

## Run simulator
User API Simulator.run

In [14]:
simulator.run(local_epochs=2)

INFO:root:*******starting rounds 1 optimization******
INFO:root:optimize the 1-th clients
INFO:root:Update Epoch: 0 	Loss: 2.803991
INFO:root:Epoch: 0	Loss: 2.547430
INFO:root:Update Epoch: 1 	Loss: 2.184564
INFO:root:Epoch: 1	Loss: 2.186581
INFO:root:optimize the 60-th clients
INFO:root:Update Epoch: 0 	Loss: 4.204228
INFO:root:Epoch: 0	Loss: 2.774040
INFO:root:Update Epoch: 1 	Loss: 2.005382
INFO:root:Epoch: 1	Loss: 2.290736
INFO:root:optimize the 88-th clients
INFO:root:Update Epoch: 0 	Loss: 3.910443
INFO:root:Update Epoch: 0 	Loss: 1.229962
INFO:root:Epoch: 0	Loss: 1.440746
INFO:root:Update Epoch: 1 	Loss: 0.730354
INFO:root:Update Epoch: 1 	Loss: 0.893425
INFO:root:Epoch: 1	Loss: 1.155814
INFO:root:optimize the 4-th clients
INFO:root:Update Epoch: 0 	Loss: 2.933777
INFO:root:Epoch: 0	Loss: 2.528071
INFO:root:Update Epoch: 1 	Loss: 1.967193
INFO:root:Epoch: 1	Loss: 2.277583
INFO:root:optimize the 69-th clients
INFO:root:Update Epoch: 0 	Loss: 4.827147
INFO:root:Epoch: 0	Loss: 3.27